In [1]:
cd ../..

/people/gerald/Documents/repositories/Educational-French-Question-Answering


In [5]:
import os
import json
import evaluate
import random
from src.data_utils.pb_corpus import FQAGPBDataset
from src.data_utils.corpus import MixedDataset, KeyMapDataset
from src.model.mbart_qg import MBARTQGCollateTokenizer

In [6]:
os.environ['EFQADATA'] = '/people/gerald/Documents/repositories/Educational-French-Question-Answering/dataset'
data_folder = os.path.expandvars("$EFQADATA/source")
datasets_name = ["squad.pb.json","fquad.pb.json"]
datasets = {}

split = "train"

for dataset_name in datasets_name: 
    with open(os.path.join(data_folder, dataset_name)) as f:
        data = json.load(f)
        datasets[dataset_name.split('.')[0]] = FQAGPBDataset(data['train'], sampler = lambda x : [x[random.randint(0, len(x) - 1)]])


In [7]:
md = KeyMapDataset(MixedDataset(*datasets.values()))

In [8]:
md[0]

{'is_default': True,
 'id': 0,
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to <hl>Saint Bernadette Soubirous<hl> in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'question_type': 'NONE',
 'input_lang': 250008,
 'output_lang': 250008,
 'index': 0,
 'dataset_index': 0}

In [9]:
from torch.utils.data import DataLoader

In [16]:
dl  = DataLoader(md, batch_size = 2, shuffle=True)

In [17]:
next(iter(dl))

{'is_default': tensor([True, True]),
 'id': tensor([39612, 36479]),
 'question': ['Who is the mascot of Northwestern Athletics?',
  'Who marketed epinephrine?'],
 'context': ['The mascot of Northwestern Athletics is <hl>Willie the Wildcat<hl>. The first mascot, however, was a live, caged bear cub from the Lincoln Park Zoo named Furpaw who was brought to the playing field on the day of a game to greet the fans. But after a losing season, the team, deciding that Furpaw was to blame for its misfortune, banished him from campus forever. Willie the Wildcat made his debut in 1933 first as a logo, and then in three dimensions in 1947, when members of the Alpha Delta fraternity dressed as wildcats during a Homecoming Parade. The Northwestern University Marching Band (NUMB) performs at all home football games and leads cheers in the student section and performs the Alma Mater at the end of the game.',
  'By the 1890s the profound effect of adrenal extracts on many different tissue types had bee

In [18]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")


In [22]:
class TMBARTQGCollateTokenizer():
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer
    
    def __call__(self, batch : dict) -> dict:
        print({for i in batch)
        source = self.tokenizer(batch['context'], return_tensors="pt",  padding='longest', truncation=True, max_length=512)
        target = self.tokenizer(batch['question'], return_tensors="pt",  padding='longest', truncation=True, max_length=512)

        source_input_ids = source.input_ids
        target_input_ids = target.input_ids

        source_input_ids[:, 0] = batch["input_lang"]
        target_input_ids[:, 0] = batch["output_lang"]
        
        return {
            "input_ids": source_input_ids,
            "attention_mask": source.attention_mask,
            "labels": target_input_ids
        }


In [23]:
dl  = DataLoader(md, batch_size = 2, shuffle=True, collate_fn=TMBARTQGCollateTokenizer(tokenizer))

In [24]:
next(iter(dl))

[{'is_default': True, 'id': 64640, 'question': "In addition to the head and abdomen, what is the other major section of an insect's body?", 'context': 'Insects (from Latin insectum, a calque of Greek ἔντομον [éntomon], "cut into sections") are a class of invertebrates within the arthropod phylum that have a chitinous exoskeleton, a three-part body (head, <hl>thorax<hl> and abdomen), three pairs of jointed legs, compound eyes and one pair of antennae. They are the most diverse group of animals on the planet, including more than a million described species and representing more than half of all known living organisms. The number of extant species is estimated at between six and ten million, and potentially represent over 90% of the differing animal life forms on Earth. Insects may be found in nearly all environments, although only a small number of species reside in the oceans, a habitat dominated by another arthropod group, crustaceans.', 'question_type': 'NONE', 'input_lang': 250008, '

TypeError: list indices must be integers or slices, not str

In [19]:
collate_fn  = MBARTQGCollateTokenizer(tokenizer)

In [20]:
collate_fn(next(iter(dl)))

{'input_ids': tensor([[250008,     62,   1202,   2271,  69988,     13,   5426,     83, 137567,
           77546,  11034,   3501,     10,   1202,  26551,    136,      8,  13158,
            4778, 117396, 130732,  31075,      5,     62,   1202,   2271,  69988,
              13,     15,    289,    991,  51529,    237,     10,   4426,   8266,
            2740,  10248,   3025,   2271,  69988,     13,  16093,   8266,   2740,
              16, 114864,   6626,    707,   1286,  31913,      7,     47,      6,
          127219,     67,   1829,   6867,      4, 134477,     13,  10821,   6626,
               9,   7514,   1202,    136,  18215, 179965,      7,      5,   3293,
              83,  27983, 163846,    297,    390,     70,   4527,    111,     10,
            6024,  38496,   6226,  25072,     15,     11,   9879,  29367, 113068,
             136,  11782,  24365,   5426,     16,    707,    390,     10,  21373,
             351,      9,  58905,  29367,   6024,  38496,   3540,  41159,      6,
   